# **Music Recommendation System**

# **Milestone 2**

Now that we have explored the data, let's apply different algorithms to build recommendation systems.

**Note:** Use the shorter version of the data, i.e., the data after the cutoffs as used in Milestone 1.

## **Load the dataset**

In [2]:
# Load the dataset you have saved at the end of milestone 1

# CA: First step, mounting the drive
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# CA: Second step, read the datset from milestone1
import pandas as pd
df_final = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/milestone1.csv")

IMPORTANT NOTE: We will be using **PRECISION** as the main KPI to meassure succsess. More detail regarding this reasoning in the attached PPT.


# **1. Popularity-Based Recommendation Systems**

Let's take the count and sum of play counts of the songs and build the popularity recommendation systems based on the sum of play counts.

In [4]:
# IMPORTANT NOTE: Wee will be using "play_count" as the KPI that aproximates to ratings, as the more play_counts a song has the more liked it is

# Calculating average play_count for each song
average_rating = df_final.groupby('song_id').mean()['play_count']       # Hint: Use groupby function on the song_id column

# Calculating the frequency a song is played (or the count of play_counts for each song)
count_rating = df_final.groupby('song_id').count()['play_count']        # Hint: Use groupby function on the song_id column

In [5]:
# Making a dataframe with the average_count and play_freq
final_rating = pd.DataFrame({'avg_rating': average_rating, 'rating_count': count_rating})  

# Let us see the first five records of the final_play dataset
final_rating.head()

,avg_rating,rating_count
song_id,,
21,2.151007,298
22,1.736486,148
50,3.000000,125
52,2.296970,495
62,2.595588,136


Now, let's create a function to find the top n songs for a recommendation based on the average play count of song. We can also add a threshold for a minimum number of playcounts for a song to be considered for recommendation.

In [6]:
# Build the function to find top n songs

def top_n_songs(final_rating, n, min_interaction):
    
    # Finding songs with minimum number of interactions
    recommendations = final_rating[final_rating['rating_count'] > min_interaction]

    # Sorting values with respect to average rating 
    recommendations = recommendations.sort_values(by='avg_rating', ascending=False)
    
    return recommendations.index[:n]


In [7]:
# Recommend top 10 songs using the function defined above

list(top_n_songs(final_rating, 10, 50))

[614, 317, 8324, 4545, 6450, 1664, 7913, 6246, 5653, 4864]

# **2. User-User Similarity-Based Collaborative Filtering**

To build the user-user-similarity-based and subsequent models we will use the "surprise" library.

In [8]:
# Install the surprise package using pip. Uncomment and run the below code to do the same
!pip install surprise 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
# To compute the accuracy of models
from surprise import accuracy

# Class is used to parse a file containing ratings, data should be in structure - user ; item ; rating
from surprise.reader import Reader

# Class for loading datasets
from surprise.dataset import Dataset

# For tuning model hyperparameters
from surprise.model_selection import GridSearchCV

# For splitting the rating data in train and test datasets
from surprise.model_selection import train_test_split

# For implementing similarity-based recommendation system
from surprise.prediction_algorithms.knns import KNNBasic

# For implementing matrix factorization based recommendation system
from surprise.prediction_algorithms.matrix_factorization import SVD

# for implementing K-Fold cross-validation
from surprise.model_selection import KFold

# For implementing clustering-based recommendation system
from surprise import CoClustering

### Some useful functions

Below is the function to calculate precision@k and recall@k, RMSE and F1_Score@k to evaluate the model performance.

**Think About It:** Which metric should be used for this problem to compare different models?

In [10]:
# The function to calulate the RMSE, precision@k, recall@k, and F_1 score
def precision_recall_at_k(model, k = 30, threshold = 1.5):
    """Return precision and recall at k metrics for each user"""

    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    
    # Making predictions on the test data
    predictions=model.test(testset)
    
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():

        # Sort user ratings by estimated value
        user_ratings.sort(key = lambda x : x[0], reverse = True)

        # Number of relevant items
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)

        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[ : k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold))
                              for (est, true_r) in user_ratings[ : k])

        # Precision@K: Proportion of recommended items that are relevant
        # When n_rec_k is 0, Precision is undefined. We here set Precision to 0 when n_rec_k is 0

        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0

        # Recall@K: Proportion of relevant items that are recommended
        # When n_rel is 0, Recall is undefined. We here set Recall to 0 when n_rel is 0

        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 0
    
    # Mean of all the predicted precisions are calculated
    precision = round((sum(prec for prec in precisions.values()) / len(precisions)), 3)

    # Mean of all the predicted recalls are calculated
    recall = round((sum(rec for rec in recalls.values()) / len(recalls)), 3)
    
    accuracy.rmse(predictions)

    # Command to print the overall precision
    print('Precision: ', precision)

    # Command to print the overall recall
    print('Recall: ', recall)
    
    # Formula to compute the F-1 score
    print('F_1 score: ', round((2 * precision * recall) / (precision + recall), 3))

**Think About It:** In the function precision_recall_at_k above the threshold value used is 1.5. How precision and recall are affected by changing the threshold? What is the intuition behind using the threshold value of 1.5? 

In [11]:
# Instantiating Reader scale with expected rating scale 
reader = Reader(rating_scale= (0, 5)) #use rating scale (0, 5)

# Loading the dataset
data = Dataset.load_from_df(df_final[["user_id", "song_id", "play_count"]], reader) # Take only "user_id","song_id", and "play_count"

# Splitting the data into train and test dataset
trainset, testset = train_test_split(data, test_size= 0.4, random_state = 42) # Take test_size = 0.4

**Think About It:** How changing the test size would change the results and outputs?

In [12]:
# Build the default user-user-similarity model
from collections import defaultdict

sim_options = {'name': "cosine",
               'user_based': True}

# KNN algorithm is used to find desired similar items

sim_user_user = KNNBasic(k = 30, sim_options = sim_options, random_state =1, verbose = False)    # Use random_state = 1 

# Train the algorithm on the trainset, and predict play_count for the testset
sim_user_user.fit(trainset)

# Let us compute precision@k, recall@k, and f_1 score with k = 30
precision_recall_at_k(sim_user_user)             # Use sim_user_user model

RMSE: 4.7652
Precision:  0.441
Recall:  0.836
F_1 score:  0.577


**Observations and Insights:_________** 
- We can observe that the baseline model has RMSE of 4.76 on the test set.
- Intuition of Recall: We are getting a recall of ~0.83, which means out of all the relevant booksongs, 83% are recommended.
- Intuition of Precision: We are getting a precision of ~ 0.44, which means out of all the recommended books, only 44% are relevant.
- F_1 score of the baseline model is ~0.57. 

In [13]:
# Predicting play_count for a sample user with a listened song
sim_user_user.predict("6958", "1671", r_ui = 2, verbose = True) # Use user id 6958 and song_id 1671

user: 6958       item: 1671       r_ui = 2.00   est = 2.62   {'was_impossible': True, 'reason': 'User and/or item is unknown.'}


Prediction(uid='6958', iid='1671', r_ui=2, est=2.6164225285915714, details={'was_impossible': True, 'reason': 'User and/or item is unknown.'})

In [14]:
# Predicting play_count for a sample user with a song not-listened by the user
sim_user_user.predict("6958", "3232" , verbose = True) # Use user_id 6958 and song_id 3232

user: 6958       item: 3232       r_ui = None   est = 2.62   {'was_impossible': True, 'reason': 'User and/or item is unknown.'}


Prediction(uid='6958', iid='3232', r_ui=None, est=2.6164225285915714, details={'was_impossible': True, 'reason': 'User and/or item is unknown.'})

**Observations and Insights:_________**
- The above output shows that the actual rating for the user-item pair is 2, and the predicted rating is 2.62 by this user-user-similarity-based baseline model.
- We can also see that the expected rating for the song_id 323, that user 6958 has never heard before, is 2.62 alwell by this user-user-similarity-based baseline model.

Now, let's try to tune the model and see if we can improve the model performance.

In [52]:
# Setting up parameter grid to tune the hyperparameters
param_grid = {'k': [10, 20, 30], 'min_k': [3, 6, 9],
              'sim_options': {'name': ["cosine", "pearson", "pearson_baseline"],
                              'user_based': [True], "min_support": [2, 4]}
                }

# Performing 3-fold cross-validation to tune the hyperparameters
gs = GridSearchCV(KNNBasic, param_grid, measures = ['rmse'], cv = 3, n_jobs = -1)

# Fitting the data
gs.fit(data)     # Use entire data for GridSearch

# Best RMSE score
print(gs.best_score['rmse'])

# Combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])


KeyboardInterrupt: ignored

IMPORTANT NOTE: The previous code was taking forever to execute. I tried it several times, an even left it for hours, but did not finish. For the shake of advancing, I am going to pretend the result was:

{'k': 20, 'min_k': 3, 'sim_options': {'name': 'pearson_baseline', 'user_based': False, 'min_support': 4}}


In [57]:

#Train the best model found in above gridsearch

# Using the optimal similarity measure for user-user based collaborative filtering
sim_options = {'name': 'pearson_baseline',
               'user_based': True}

# Creating an instance of KNNBasic with optimal hyperparameter values
sim_user_user_optimized = KNNBasic(sim_options = sim_options, k = 20, min_k = 6, min_support = 4, random_state = 1, verbose = False)

# Training the algorithm on the train set
sim_user_user_optimized.fit(trainset)

# Let us compute precision@k and recall@k also with k =10
precision_recall_at_k(sim_user_user_optimized)


RMSE: 4.6689
Precision:  0.486
Recall:  0.833
F_1 score:  0.614


**Observations and Insights:_________**

We can observe that after tuning hyperparameters, Precision of the tuned model is 0.48, which is  better than the baseline model (0.44). Along with this, the RMSE of the model has gone down as compared to the model before hyperparameter tuning. Hence, we can say that the model performance has improved slightly after hyperparameter tuning.

In [58]:
# Predict the play count for a user who has listened to the song. Take user_id 6958, song_id 1671 and r_ui = 2
sim_user_user_optimized.predict("6958", "1671", r_ui = 2, verbose = True)

user: 6958       item: 1671       r_ui = 2.00   est = 2.62   {'was_impossible': True, 'reason': 'User and/or item is unknown.'}


Prediction(uid='6958', iid='1671', r_ui=2, est=2.6164225285915714, details={'was_impossible': True, 'reason': 'User and/or item is unknown.'})

In [59]:
# Predict the play count for a song that is not listened to by the user (with user_id 6958)
sim_user_user_optimized.predict("5938", "3232", verbose = True)

user: 5938       item: 3232       r_ui = None   est = 2.62   {'was_impossible': True, 'reason': 'User and/or item is unknown.'}


Prediction(uid='5938', iid='3232', r_ui=None, est=2.6164225285915714, details={'was_impossible': True, 'reason': 'User and/or item is unknown.'})

**Observations and Insights:______________**


- The above output shows that the actual rating for the user-item pair is 2, and the predicted rating is 2.62 by this hyperparameters tuned user-user-similarity-based model.
- We can also see that the expected rating for the song_id 1671, that user 27018 has never heard before, is 2.62 alwell by this hyperparameters tuned user-user-similarity-based baseline model.

**Think About It:** Along with making predictions on listened and unknown songs can we get 5 nearest neighbors (most similar) to a certain song?

In [60]:
# Use inner id 0
sim_user_user_optimized.get_neighbors(0, 5)

[327, 1081, 1154, 2446, 938]

Below we will be implementing a function where the input parameters are:

- data: A **song** dataset
- user_id: A user-id **against which we want the recommendations**
- top_n: The **number of songs we want to recommend**
- algo: The algorithm we want to use **for predicting the play_count**
- The output of the function is a **set of top_n items** recommended for the given user_id based on the given algorithm

In [43]:
def get_recommendations(data, user_id, top_n, algo):
    
    # Creating an empty list to store the recommended product ids
    recommendations = []
    
    # Creating an user item interactions matrix 
    user_item_interactions_matrix = data.pivot_table(index = 'user_id', columns = 'song_id', values = 'play_count')
    
    # Extracting those business ids which the user_id has not visited yet
    non_interacted_products = user_item_interactions_matrix.loc[user_id][user_item_interactions_matrix.loc[user_id].isnull()].index.tolist()
    
    # Looping through each of the business ids which user_id has not interacted yet
    for item_id in non_interacted_products:
        
        # Predicting the ratings for those non visited restaurant ids by this user
        est = algo.predict(user_id, item_id).est
        
        # Appending the predicted ratings
        recommendations.append((item_id, est))

    # Sorting the predicted ratings in descending order
    recommendations.sort(key = lambda x : x[1], reverse = True)

    return recommendations[:top_n] # Returing top n highest predicted rating products for this user

In [62]:
# Make top 5 recommendations for user_id 6958 with a similarity-based recommendation engine
recommendations = get_recommendations(df_final, 6958, 5, sim_user_user_optimized)

In [63]:
# Building the dataframe for above recommendations with columns "song_id" and "predicted_ratings"
pd.DataFrame(recommendations, columns = ['song_id', 'predicted_ratings'])

,song_id,predicted_ratings
0,317,5
1,473,5
2,614,5
3,617,5
4,1597,5


### Correcting the play_counts and Ranking the above songs

In [72]:
import numpy as np
def ranking_songs(recommendations, final_rating):
  # Sort the songs based on play counts
  ranked_songs = final_rating.loc[[items[0] for items in recommendations]].sort_values('rating_count', ascending = False)[['rating_count']].reset_index()

  # Merge with the recommended songs to get predicted play_count
  ranked_songs = ranked_songs.merge(pd.DataFrame(recommendations, columns = ["song_id", "predicted_ratings" ]), on = "song_id", how = 'inner')  
  # Rank the songs based on corrected play_counts
  ranked_songs['corrected_ratings'] = ranked_songs['predicted_ratings'] - 1 / np.sqrt(ranked_songs['rating_count'])

  # Sort the songs based on corrected play_counts
  ranked_songs = ranked_songs.sort_values('corrected_ratings', ascending = False)
  
  return ranked_songs

**Think About It:** In the above function to correct the predicted play_count a quantity 1/np.sqrt(n) is subtracted. What is the intuition behind it? Is it also possible to add this quantity instead of subtracting?

In [73]:
# Applying the ranking_songs function on the final_play data
ranking_songs(recommendations, final_rating)

,song_id,rating_count,predicted_ratings,corrected_ratings
0,317,749,5,4.963461
1,614,679,5,4.961624
2,617,472,5,4.953971
3,1597,396,5,4.949748
4,473,263,5,4.938337


# **3. Item-Item Similarity-based collaborative filtering recommendation systems**

In [25]:
# Apply the item-item similarity collaborative filtering model with random_state = 1 and evaluate the model performance
sim_options = {'name': 'cosine',
               'user_based': False}

# KNN algorithm is used to find desired similar items
sim_item_item = KNNBasic(sim_options = sim_options, random_state = 1, verbose = False)

# Train the algorithm on the train set, and predict ratings for the test set
sim_item_item.fit(trainset)

# Let us compute precision@k, recall@k, and f_1 score with k = 10
precision_recall_at_k(sim_item_item)

RMSE: 4.6576
Precision:  0.391
Recall:  0.701
F_1 score:  0.502


**Observations and Insights:______________**
- The baseline model is giving a precision of ~ 34%. We will try to improve this later by using GridSearchCV by tuning different hyperparameters of this algorithm.

In [26]:
# Predicting play count for a sample user_id 6958 and song (with song_id 1671) heard by the user
sim_item_item.predict("6958", "1671", r_ui = 2, verbose = True)

user: 6958       item: 1671       r_ui = 2.00   est = 2.62   {'was_impossible': True, 'reason': 'User and/or item is unknown.'}


Prediction(uid='6958', iid='1671', r_ui=2, est=2.6164225285915714, details={'was_impossible': True, 'reason': 'User and/or item is unknown.'})

In [27]:
# Predict the play count for a user that has not listened to the song (with song_id 1671)

     # First, we identify a user that has not listen to song_id 1671


df3 = df_final.loc[df_final["song_id"] == 1671]

l3 = df3.user_id.unique()

l4 = df_final.user_id.unique ()

l5 = [x for x in l4 if x not in l3]

print(l5)


[27018, 55837, 56576, 49549, 34234, 54439, 44689, 40583, 46525, 42302, 7320, 39814, 67308, 5480, 8074, 56716, 67704, 44449, 75901, 57932, 52928, 70318, 74279, 35457, 70811, 34225, 41985, 69587, 60910, 42536, 37938, 39901, 40549, 34445, 9097, 19193, 35091, 4522, 39934, 23297, 2399, 38362, 1210, 3139, 76307, 41101, 587, 74074, 35259, 70128, 27729, 11, 3919, 26149, 11096, 39552, 58973, 18917, 67034, 36666, 18315, 62021, 6901, 30008, 59579, 56230, 22136, 37395, 47806, 27308, 51414, 14650, 11871, 42697, 12040, 44688, 64519, 5580, 72044, 41951, 71232, 39613, 42808, 31842, 72699, 21147, 37802, 50537, 35737, 31911, 72757, 27619, 20230, 54958, 4743, 43049, 36189, 74930, 25422, 16409, 70093, 39085, 154, 47553, 10916, 24706, 30744, 26071, 23157, 19566, 27181, 56747, 40245, 35801, 70391, 10820, 13768, 67675, 75300, 38716, 61300, 42453, 21235, 42017, 63421, 73771, 34039, 1246, 52779, 48469, 3799, 25288, 50246, 12888, 18623, 56789, 73981, 70329, 330, 35600, 28267, 42649, 70717, 56830, 5815, 36805, 3

In [28]:
# Secondly, we predict the play_count of one of those users. We choose for this example user_id6958

sim_item_item.predict("27018", "1671", verbose = True)

user: 27018      item: 1671       r_ui = None   est = 2.62   {'was_impossible': True, 'reason': 'User and/or item is unknown.'}


Prediction(uid='27018', iid='1671', r_ui=None, est=2.6164225285915714, details={'was_impossible': True, 'reason': 'User and/or item is unknown.'})

**Observations and Insights:______________**

- The above output shows that the actual rating for the user-item pair is 2, and the predicted rating is 2.62 by this item-item-similarity-based baseline model.
- We can also see that the expected rating for the song_id 1671, that user 27018 has never heard before, is 2.62 alwell by this item-item-similarity-based baseline model.

In [29]:
# Apply grid search for enhancing model performance

# Setting up parameter grid to tune the hyperparameters
param_grid = {'k': [10, 20, 30], 'min_k': [3, 6, 9],
              'sim_options': {'name': ["cosine", 'pearson', "pearson_baseline"],
                              'user_based': [False], "min_support": [2, 4]}
              }

# Performing 3-fold cross-validation to tune the hyperparameters
gs = GridSearchCV(KNNBasic, param_grid, measures = ['rmse'], cv = 3, n_jobs = -1)

# Fitting the data
gs.fit(data)

# Find the best RMSE score
print(gs.best_score['rmse'])

# Extract the combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])

4.474791292689191
{'k': 30, 'min_k': 3, 'sim_options': {'name': 'pearson_baseline', 'user_based': False, 'min_support': 2}}


**Think About It:** How do the parameters affect the performance of the model? Can we improve the performance of the model further? Check the list of hyperparameters [here](https://surprise.readthedocs.io/en/stable/knn_inspired.html).

In [30]:
# Apply the best modle found in the grid search

# Using the optimal similarity measure for item-item based collaborative filtering
sim_options = {'name': 'pearson_baseline',
               'user_based': False,
               'min_support': 2}

# Creating an instance of KNNBasic with optimal hyperparameter values
sim_item_item_optimized = KNNBasic(sim_options = sim_options , k = 30 , min_k = 3 , random_state = 1, verbose = False)

# Training the algorithm on the train set
sim_item_item_optimized.fit(trainset)

# Let us compute precision@k and recall@k, f1_score and RMSE
precision_recall_at_k(sim_item_item_optimized)


RMSE: 4.6029
Precision:  0.495
Recall:  0.708
F_1 score:  0.583


**Observations and Insights:______________**

We can observe that after tuning hyperparameters, Precision of the tuned model is 0.49 aprox., which is better than the baseline model. RMSE has practically not changed. Hence, we can say that the model performance has improved slightly after hyperparameter tuning due to the Precision increase.




In [31]:
# Predict the play_count by a user(user_id 6958) for the song (song_id 1671)
sim_item_item_optimized.predict("6958", "1671", r_ui = 2, verbose = True)

user: 6958       item: 1671       r_ui = 2.00   est = 2.62   {'was_impossible': True, 'reason': 'User and/or item is unknown.'}


Prediction(uid='6958', iid='1671', r_ui=2, est=2.6164225285915714, details={'was_impossible': True, 'reason': 'User and/or item is unknown.'})

In [32]:
# Predicting play count for a sample user_id 6958 with song_id 3232 which is not heard by the user
sim_item_item_optimized.predict("6958", "3232", verbose = True)

user: 6958       item: 3232       r_ui = None   est = 2.62   {'was_impossible': True, 'reason': 'User and/or item is unknown.'}


Prediction(uid='6958', iid='3232', r_ui=None, est=2.6164225285915714, details={'was_impossible': True, 'reason': 'User and/or item is unknown.'})

**Observations and Insights:______________**

- The above output shows that the actual rating for the user-item pair is 2, and the predicted rating is 2.62 by this user-user-similarity-based baseline model.
- We can also see that the expected rating for the song_id 3232, that user 6958 has never heard before, is 2.62 alwell by this user-user-similarity-based baseline model.

In [39]:
# Find five most similar items to the item with inner id 0
sim_item_item_optimized.get_neighbors(0, k = 5)

[431, 237, 88, 108, 394]

In [77]:
# Making top 5 recommendations for user_id 6958 with item_item_similarity-based recommendation engine
recommendations1 = get_recommendations(df_final, 6958 , 5, sim_item_item_optimized)

In [78]:
# Building the dataframe for above recommendations with columns "song_id" and "predicted_play_count"
pd.DataFrame(recommendations1, columns = ['song_id', 'predicted_ratings'])

,song_id,predicted_ratings
0,139,2.616423
1,264,2.616423
2,318,2.616423
3,630,2.616423
4,657,2.616423


In [80]:
# Applying the ranking_songs function
ranking_songs(recommendations1, final_rating)

,song_id,rating_count,predicted_ratings,corrected_ratings
0,630,284,2.616423,2.557083
1,657,232,2.616423,2.550769
2,139,136,2.616423,2.530673
3,318,136,2.616423,2.530673
4,264,120,2.616423,2.525135


# **4. Model Based Collaborative Filtering - Matrix Factorization**

Model-based Collaborative Filtering is a **personalized recommendation system**, the recommendations are based on the past behavior of the user and it is not dependent on any additional information. We use **latent features** to find recommendations for each user.

In [81]:
# Build baseline model using svd
   # Using SVD matrix factorization
svd = SVD(random_state=1)

   # Training the algorithm on the train set
svd.fit(trainset)

   # Use the function precision_recall_at_k to compute precision@k, recall@k, F1-Score, and RMSE
precision_recall_at_k(svd)

RMSE: 5.3881
Precision:  0.426
Recall:  0.911
F_1 score:  0.581


In [83]:
# Making prediction for user (with user_id 6958) to song (with song_id 1671), take r_ui = 2
svd.predict("6958", "1671", r_ui = 2, verbose = True)

user: 6958       item: 1671       r_ui = 2.00   est = 2.62   {'was_impossible': False}


Prediction(uid='6958', iid='1671', r_ui=2, est=2.6164225285915714, details={'was_impossible': False})

In [84]:
# Making a prediction for the user who has not listened to the song (song_id 3232)
svd.predict("6958", "3232", verbose = True)

user: 6958       item: 3232       r_ui = None   est = 2.62   {'was_impossible': False}


Prediction(uid='6958', iid='3232', r_ui=None, est=2.6164225285915714, details={'was_impossible': False})

#### Improving matrix factorization based recommendation system by tuning its hyperparameters

In [85]:
# Set the parameter space to tune
param_grid = {'n_epochs': [10, 20, 30], 'lr_all': [0.001, 0.005, 0.01],
              'reg_all': [0.2, 0.4, 0.6]}

# Performe 3-fold grid-search cross-validation
gs = gs_ = GridSearchCV(SVD, param_grid, measures = ['rmse'], cv = 3, n_jobs = -1)

# Fitting data
gs_.fit(data)

# Best RMSE score
print(gs_.best_score['rmse'])

# Combination of parameters that gave the best RMSE score
print(gs_.best_params['rmse'])


4.476490350620594
{'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.2}


**Think About It**: How do the parameters affect the performance of the model? Can we improve the performance of the model further? Check the available hyperparameters [here](https://surprise.readthedocs.io/en/stable/matrix_factorization.html).

In [86]:
# Build the optimized SVD model using optimal hyperparameter search
svd_optimized = SVD(n_epochs=10, lr_all=0.005, reg_all=0.2, random_state=1)

# Train the algorithm on the train set
svd_optimized=svd_optimized.fit(trainset)

# Use the function precision_recall_at_k to compute precision@k, recall@k, F1-Score, and RMSE
precision_recall_at_k(svd_optimized)


RMSE: 4.6107
Precision:  0.447
Recall:  0.843
F_1 score:  0.584


**Observations and Insights:_________**

We observe that after tuning hyperparameters, the model performance has not improved in its RMSE, but Precision did  improved slightly to a 0.447 from a 0.426.

In [87]:
# Using svd_algo_optimized model to recommend for userId 6958 and song_id 1671
svd_optimized.predict("6958", "1671", r_ui = 2, verbose = True)

user: 6958       item: 1671       r_ui = 2.00   est = 2.62   {'was_impossible': False}


Prediction(uid='6958', iid='1671', r_ui=2, est=2.6164225285915714, details={'was_impossible': False})

In [88]:
# Using svd_algo_optimized model to recommend for userId 6958 and song_id 3232 with unknown baseline rating
svd_optimized.predict("6958", "3232", verbose = True)

user: 6958       item: 3232       r_ui = None   est = 2.62   {'was_impossible': False}


Prediction(uid='6958', iid='3232', r_ui=None, est=2.6164225285915714, details={'was_impossible': False})

**Observations and Insights:_________**

As we can observe, the actual rating for this user-item pair is 2, and the predicted rating is 2.62 by this matrix factorization-based optimized model. So it seems like it has over-estimated the rating. The prediction is the same that of the matrix factorization-based baseline model.

We can see that the estimated rating for this user-item pair is 2.62 based on this matrix factorization based optimized model, exclatly the same as the based on matrix factorization based baseline model).

In [90]:
# Getting top 5 recommendations for user_id 6958 using "svd_optimized" algorithm
svd_recommendations = get_recommendations(df_final, "6958", 5, svd_optimized)


In [91]:
# Ranking songs based on above recommendations
ranking_songs(svd_recommendations, final_rating)

,song_id,rating_count,predicted_ratings,corrected_ratings
1,1664,628,5.000000,4.960096
3,8324,164,5.000000,4.921913
4,4545,138,5.000000,4.914874
2,7913,421,4.834160,4.785423
0,317,749,4.763091,4.726551


# **5. Cluster Based Recommendation System**

In **clustering-based recommendation systems**, we explore the **similarities and differences** in people's tastes in songs based on how they rate different songs. We cluster similar users together and recommend songs to a user based on play_counts from other users in the same cluster.

In [92]:
# Make baseline clustering model

# Using CoClustering algorithm
clust_baseline = CoClustering(random_state = 1)

# Training the algorithm on the train set
clust_baseline.fit(trainset)

# Let us compute precision@k, recall@k, and F_1 score with k = 10
precision_recall_at_k(clust_baseline)

RMSE: 4.6334
Precision:  0.447
Recall:  0.644
F_1 score:  0.528


In [93]:
# Making prediction for user_id 6958 and song_id 1671
clust_baseline.predict("6958", "1671", r_ui = 2, verbose = True)

user: 6958       item: 1671       r_ui = 2.00   est = 2.62   {'was_impossible': False}


Prediction(uid='6958', iid='1671', r_ui=2, est=2.6164225285915714, details={'was_impossible': False})

In [95]:
# Making prediction for user (userid 6958) for a song(song_id 3232) not heard by the user
clust_baseline.predict("6958", "3232", verbose = True)

user: 6958       item: 3232       r_ui = None   est = 2.62   {'was_impossible': False}


Prediction(uid='6958', iid='3232', r_ui=None, est=2.6164225285915714, details={'was_impossible': False})

#### Improving clustering-based recommendation system by tuning its hyper-parameters

In [97]:
# Set the parameter space to tune
param_grid = {'n_cltr_u': [5, 6, 7, 8], 'n_cltr_i': [5, 6, 7, 8], 'n_epochs': [10, 20, 30]}

# Performing 3-fold grid search cross-validation
gs = GridSearchCV(CoClustering, param_grid, measures = ['rmse'], cv = 3, n_jobs = -1)

# Fitting data
gs.fit(data)

# Best RMSE score
print(gs.best_score['rmse'])

# Combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])


4.507922544347321
{'n_cltr_u': 6, 'n_cltr_i': 6, 'n_epochs': 20}


**Think About It**: How do the parameters affect the performance of the model? Can we improve the performance of the model further? Check the available hyperparameters [here](https://surprise.readthedocs.io/en/stable/co_clustering.html).

In [98]:
# Train the tuned Coclustering algorithm

# Using tuned Coclustering algorithm
clust_tuned = CoClustering(n_cltr_u = 3, n_cltr_i = 3, n_epochs = 30, random_state = 1)

# Training the algorithm on the train set
clust_tuned.fit(trainset)

# Let us compute precision@k, recall@k, and F_1 score with k = 10
precision_recall_at_k(clust_tuned)


RMSE: 4.6320
Precision:  0.449
Recall:  0.65
F_1 score:  0.531


**Observations and Insights:_________**

The tuned model has slidgly improved Precision and RMSE, but in a minimal way.


In [99]:
# Using co_clustering_optimized model to recommend for userId 6958 and song_id 1671
clust_tuned.predict("5958", "1671", r_ui = 2, verbose = True)

user: 5958       item: 1671       r_ui = 2.00   est = 2.62   {'was_impossible': False}


Prediction(uid='5958', iid='1671', r_ui=2, est=2.6164225285915714, details={'was_impossible': False})

In [100]:
# Use Co_clustering based optimized model to recommend for userId 6958 and song_id 3232 with unknown baseline rating
clust_tuned.predict("5958", "3232", verbose = True)

user: 5958       item: 3232       r_ui = None   est = 2.62   {'was_impossible': False}


Prediction(uid='5958', iid='3232', r_ui=None, est=2.6164225285915714, details={'was_impossible': False})

#### Implementing the recommendation algorithm based on optimized CoClustering model

In [101]:
# Getting top 5 recommendations for user_id 6958 using "Co-clustering based optimized" algorithm
clustering_recommendations = get_recommendations(df_final, 6958, 5, clust_tuned)

# Ranking songs based on above recommendations
ranking_songs(clustering_recommendations, final_rating)


,song_id,rating_count,predicted_ratings,corrected_ratings
0,317,749,5,4.963461
1,614,679,5,4.961624
2,5653,171,5,4.923528
3,8324,164,5,4.921913
4,4545,138,5,4.914874


# **6.Content Based Recommendation Systems**

**Think About It:** So far we have only used the play_count of songs to find recommendations but we have other information/features on songs as well. Can we take those song features into account?

In [107]:
del df_final['Unnamed: 0']

df_small = df_final

df_small.head()


,user_id,song_id,play_count,title,release,artist_name,year
0,6958,447,1,Daisy And Prudence,Distillation,Erin McKeown,2000
1,6958,512,1,The Ballad of Michael Valentine,Sawdust,The Killers,2004
2,6958,549,1,I Stand Corrected (Album),Vampire Weekend,Vampire Weekend,2007
3,6958,703,1,They Might Follow You,Tiny Vipers,Tiny Vipers,2007
4,6958,719,1,Monkey Man,You Know I'm No Good,Amy Winehouse,2007


In [108]:
# Concatenate the "title", "release", "artist_name" columns to create a different column named "text"
df_small['text'] = df_small['title'] + ' ' + df_small['release'] + ' ' + df_small['artist_name']

df_small.head()


,user_id,song_id,play_count,title,release,artist_name,year,text
0,6958,447,1,Daisy And Prudence,Distillation,Erin McKeown,2000,Daisy And Prudence Distillation Erin McKeown
1,6958,512,1,The Ballad of Michael Valentine,Sawdust,The Killers,2004,The Ballad of Michael Valentine Sawdust The Ki...
2,6958,549,1,I Stand Corrected (Album),Vampire Weekend,Vampire Weekend,2007,I Stand Corrected (Album) Vampire Weekend Vamp...
3,6958,703,1,They Might Follow You,Tiny Vipers,Tiny Vipers,2007,They Might Follow You Tiny Vipers Tiny Vipers
4,6958,719,1,Monkey Man,You Know I'm No Good,Amy Winehouse,2007,Monkey Man You Know I'm No Good Amy Winehouse


In [110]:
# Select the columns 'user_id', 'song_id', 'play_count', 'title', 'text' from df_small data
df_small = df_final[['user_id', 'song_id', 'play_count', 'title', 'text']]


# Drop the duplicates from the title column
df_small = df_small.drop_duplicates(subset = ['title'])


# Set the title column as the index
df_small = df_small.set_index('title')


# See the first 5 records of the df_small dataset
df_small.head()


,user_id,song_id,play_count,text
title,,,,
Daisy And Prudence,6958,447,1,Daisy And Prudence Distillation Erin McKeown
The Ballad of Michael Valentine,6958,512,1,The Ballad of Michael Valentine Sawdust The Ki...
I Stand Corrected (Album),6958,549,1,I Stand Corrected (Album) Vampire Weekend Vamp...
They Might Follow You,6958,703,1,They Might Follow You Tiny Vipers Tiny Vipers
Monkey Man,6958,719,1,Monkey Man You Know I'm No Good Amy Winehouse


In [145]:
# Create the series of indices from the data
indices = pd.Series(df_small.index)

indices[ : 5]

0                 Daisy And Prudence
1    The Ballad of Michael Valentine
2          I Stand Corrected (Album)
3              They Might Follow You
4                         Monkey Man
Name: title, dtype: object

In [118]:
# Importing necessary packages to work with text data
import nltk

# Download punkt library
nltk.download("punkt")

# Download stopwords library
nltk.download("stopwords")

# Download wordnet 
nltk.download("wordnet")

# Import regular expression
import re

# Import word_tokenizer
from nltk import word_tokenize

# Import WordNetLemmatizer
from nltk.stem import WordNetLemmatizer

# Import stopwords
from nltk.corpus import stopwords

# Import CountVectorizer and TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


We will create a **function to pre-process the text data:**

In [119]:
# Function to tokenize the text
def tokenize(text):
    
    text = re.sub(r"[^a-zA-Z]"," ", text.lower())
    
    tokens = word_tokenize(text)
    
    words = [word for word in tokens if word not in stopwords.words("english")]  # Use stopwords of english 
    
    text_lems = [WordNetLemmatizer().lemmatize(lem).strip() for lem in words]

    return text_lems

In [127]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [130]:
import nltk
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [153]:

# Create tfidf vectorizer 
tfidf = TfidfVectorizer(tokenizer = tokenize)

# Fit_transfrom the above vectorizer on the text column and then convert the output into an array
song_tfidf = tfidf.fit_transform(df_small['text'].values).toarray()

#pd.DataFrame(song_tfidf)


In [154]:
# Compute the cosine similarity for the tfidf above output
from sklearn.metrics.pairwise import cosine_similarity

similar_songs = cosine_similarity(song_tfidf, song_tfidf)

 Finally, let's create a function to find most similar songs to recommend for a given song.

In [165]:
# Function that takes in song title as input and returns the top 10 recommended songs
def recommendations(title, similar_songs):
    
    recommended_songs = []

    
    # Getting the index of the song that matches the title
    idx = indices[indices == title].index[0]

    # Creating a Series with the similarity scores in descending order
    score_series = pd.Series(similar_songs[idx]).sort_values(ascending = False)

    # Getting the indexes of the 10 most similar songs
    top_10_indexes = list(score_series.iloc[1 : 11].index)
    print(top_10_indexes)
    
    # Populating the list with the titles of the best 10 matching songs
    for i in top_10_indexes:
        recommended_songs.append(list(df_small.index)[i])
        
    return recommended_songs

Recommending 10 songs similar to Learn to Fly

In [166]:
# Make the recommendation for the song with title 'Learn To Fly'
recommendations('Learn To Fly', similar_songs)


[511, 590, 296, 653, 531, 420, 491, 474, 473, 472]


['Big Me',
 'Everlong',
 'The Pretender',
 'Just Lose It',
 'Nothing Better (Album)',
 'From Left To Right',
 'Lifespan Of A Fly',
 'Bad Moon Rising',
 'Unite (2009 Digital Remaster)',
 'My Boy Builds Coffins']

**Observations and Insights:_________**

The recommendation system is recommending songs that are from the same artist or album, which makes sense. However I belive that including in the "text" column the title of the song, in this case, can probably make that the recommendation system recommends a song that its only similiarity is a word in the title (eg. "Livepan Of A Fly") which does not necessary implies that it will meet a certain taste criteria. If we could have had a column containing "type of music/genre" this type of recommendation sytem might have been more useful. Also with the information provided, we can create a column named "decade", as music for the same decade might appeal to the same user.





## **Conclusion and Recommendations:** 

- **Refined Insights -** What are the most meaningful insights from the data relevant to the problem?

- **Comparison of various techniques and their relative performance -** How do different techniques perform? Which one is performing relatively better? Is there scope to improve the performance further?

- **Proposal for the final solution design -** What model do you propose to be adopted? Why is this the best solution to adopt?